# SRC physics

__Author:__ A. J. Tropiano [tropiano.4@osu.edu]<br/>
__Date:__ April 9, 2021

Generates SRC physics figures based on this paper: A. J. Tropiano, S. K. Bogner, and R. J. Furnstahl, [Phys. Rev. C __104__, 034311 (2021)](https://journals.aps.org/prc/abstract/10.1103/PhysRevC.104.034311).

_Last update:_ July 26, 2022

__To-do list:__
* Finish functions:
7. Data
8. Plot/initial and final features
9. Axes limits
10. Axes labels
11. Partial wave contributions table
12. Bar chart figure
* Add markdown to cells in section.
* Add docstrings to functions.
* Check imports.

In [2]:
# Python imports
from itertools import chain
from matplotlib.offsetbox import AnchoredText
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [ ]:
# Imports from A.T. codes
from scripts.figures import (
    set_rc_parameters, line_styles, xkcd_colors, label_lambda
)
from scripts.tools import replace_periods

## Set-up

In [ ]:
# Run this cell to turn on customized matplotlib graphics
set_rc_parameters()

In [ ]:
# Save figures in the following directory
figures_directory = '../figures/src/'

## Plotting functions

In [ ]:
def srg_transformation_ratio(
        k_0, kvnns, lamb, kmax=15.0, kmid=3.0, ntot=120, x_limits=(0,4),
        y_limits=(1e0,1e3), xlabel_size=16, ylabel_size=16, ratio_label=True,
        save=False):
    """
    """
    
    # --- Figure set-up --- #
    plt.close('all')
    f, ax = plt.subplots(figsize=(4,4))
    
    # --- Legend --- #
    ax.legend(loc='lower left', fontsize=13, frameon=True)
    
    # --- Additional labels --- #
    if ratio_label:  # Label 3S1/1S0
        ax.add_artist(
            AnchoredText(r'$\frac{^3{\rm S}_1}{^{1}{\rm S}_0}$',
                         loc='upper right', prop=dict(size=24), frameon=False)
        )

    # --- Save file --- #
    if save:
        file_name = f'srg_transformation_ratio_k_0_{k_0}_kvnns'
        for ikvnn in kvnns:
            file_name += f'_{ikvnn}'
        file_name += f'_lamb_{lamb}_kmax_{kmax}_kmid_{kmid}_ntot_{ntot}'
        file_name = replace_periods(file_name) + '.png'
        f.savefig(figures_directory + file_name)

In [ ]:
def interleave(*iters):
    """
    Take a set of lists of the same length and interleave them.
    """
    return list(chain(*zip(*iters)))

In [ ]:
# temp.

    def n_lambda_exact(self, q, contributions='total'):
        """
        Computes the exact deuteron momentum distribution using the wave
        function from the input Hamiltonian. Note, set interp = False to use
        this function.

        Parameters
        ----------
        q : float
            Momentum value [fm^-1].
        contributions : str, optional
            Option to return different contributions to the momentum
            distribution.
            1. Default is 'total' which only returns the total momentum
               distribution.
            2. Specify 'q_contributions' for I, \delta U, and 
               \delta U \delta U^\dagger along with the total.
            3. Specify 'partial_wave_ratio' for the ratio of 3S1-3S1 to full
               3S1-3D1 along with the absolute total.

        Returns
        -------
        output : float or tuple
            Pair momentum distribution [fm^3] evaluated at momentum q
            [fm^-1]. (Note, this function will return a tuple of floats if
            contributions is not 'total'.)

        """

        # Load deuteron wave function and delta_U_matrix (both unitless)
        psi_vector = self.psi
        delta_U_matrix = self.delta_U_matrix
        
        # Load bare momentum projection operator [fm^3]
        bare_operator = momentum_projection_operator(q, self.k_array,
                        self.k_weights, '3S1', smeared=False)
        
        # Term 1: 1 * n(q) * 1
        term_1 = psi_vector.T @ bare_operator @ psi_vector
        
        # Term 2: \deltaU * n(q) * 1
        term_deltaU = psi_vector.T @ delta_U_matrix @ bare_operator @ \
                      psi_vector
        
        # Term 3: 1 * n(q) * \deltaU^\dagger = Term 2
        term_deltaU *= 2
        
        # High-q term: \deltaU * n(q) * \deltaU^\dagger
        term_deltaU2 = psi_vector.T @ delta_U_matrix @ bare_operator @ \
                        delta_U_matrix.T @ psi_vector
                       
        if contributions == 'partial_wave_ratio':
            
            # Length of q_array
            ntot = self.ntot
            
            # 3S1-3S1 only
            numerator = abs( psi_vector.T[:ntot] @ ( \
                                 delta_U_matrix[:ntot, :ntot] @ \
                                 bare_operator[:ntot, :ntot] @ \
                                 delta_U_matrix.T[:ntot, :ntot] +
                                 delta_U_matrix[:ntot, ntot:] @ \
                                 bare_operator[ntot:, ntot:] @ \
                                 delta_U_matrix.T[ntot:, :ntot] ) @ \
                             psi_vector[:ntot] )
                        
            # Full 3S1-3D1 taking absolute values 
            denominator = numerator + \
                          abs( psi_vector.T[:ntot] @ ( \
                                   delta_U_matrix[:ntot, :ntot] @ \
                                   bare_operator[:ntot, :ntot] @ \
                                   delta_U_matrix.T[:ntot, ntot:] +
                                   delta_U_matrix[:ntot, ntot:] @ \
                                   bare_operator[ntot:, ntot:] @ \
                                   delta_U_matrix.T[ntot:, ntot:] ) @ \
                               psi_vector[ntot:] ) + \
                          abs( psi_vector.T[ntot:] @ ( \
                                   delta_U_matrix[ntot:, :ntot] @ \
                                   bare_operator[:ntot, :ntot] @ \
                                   delta_U_matrix.T[:ntot, :ntot] +
                                   delta_U_matrix[ntot:, ntot:] @ \
                                   bare_operator[ntot:, ntot:] @ \
                                   delta_U_matrix.T[ntot:, :ntot] ) @ \
                               psi_vector[:ntot] ) + \
                          abs( psi_vector.T[ntot:] @ ( \
                                   delta_U_matrix[ntot:, :ntot] @ \
                                   bare_operator[:ntot, :ntot] @ \
                                   delta_U_matrix.T[:ntot, ntot:] +
                                   delta_U_matrix[ntot:, ntot:] @ \
                                   bare_operator[ntot:, ntot:] @ \
                                   delta_U_matrix.T[ntot:, ntot:] ) @ \
                               psi_vector[ntot:] )
                                   
        # Add up each term for total
        total = term_1 + term_deltaU + term_deltaU2

        # Return contributions and total or just total
        if contributions == 'q_contributions':
            return total, term_1, term_deltaU, term_deltaU2
        elif contributions == 'partial_wave_ratio':
            return total, numerator/denominator
        else: # Default
            return total

In [ ]:
def deuteron_channel_contributions_bar_chart():
    """
    """
    return None

In [ ]:
def deuteron_momentum_distribution_contributions(
        kvnn, lamb, kmax=15.0, kmid=3.0, ntot=120, number_of_curves=4,
        x_limits=(0, 4), y_limits=(1e-5, 1e3), xlabel_size=16, ylabel_size=16,
        save=False):
    """
    """
    
    # --- Figure set-up --- #
    plt.close('all')
    f, ax = plt.subplots(figsize=(4,4))
    
    # --- Legend --- #
    ax.legend(loc='upper right', fontsize=14, frameon=False)
    
    # --- Save file --- #
    if save:
        file_name = ('deuteron_momentum_distribution_contributions_kvnn_'
                     f'{kvnn}_lamb_{lamb}_kmax_{kmax}_kmid_{kmid}_ntot_{ntot}'
                     f'number_of_curves_{number_of_curves}')
        file_name = replace_periods(file_name) + '.png'
        f.savefig(figures_directory + file_name)

In [ ]:
def factorization_ratios(
        k_0, k_i_values, kvnn, channel, lamb, kmax=15.0, kmid=3.0, ntot=120,
        x_limits=(0, 5), y_limits=(1e-1, 1e1), xlabel_size=16, ylabel_size=16,
        k0_lamb_label=True, save=False):
    """
    """
    
    # --- Figure set-up --- #
    plt.close('all')
    f, ax = plt.subplots(figsize=(4,4))
    
    # --- Legend --- #
    ax.legend(loc='upper right', fontsize=11, frameon=False)
    
    # --- Additional labels --- #
    if k0_lamb_label:  # Label k_0 and \lambda
        k_0_label = rf'$k_0={k_0}$' + ' fm' + r'$^{-1}$'
        lambda_label = label_lambda(lamb)
        ax.add_artist(
            AnchoredText(k_0_label + '\n' + lambda_label, loc='lower right',
                         prop=dict(size=11), frameon=False)
        )

    # --- Save file --- #
    if save:
        file_name = f'delU_factorization_ratios_k0_{k_0}_ki'
        for k_i in k_i_values:
            file_name += f'_{k_i}'
        file_name += (f'_{kvnn}_{channel}_lamb_{lamb}_kmax_{kmax}_kmid_{kmid}'
                      f'_ntot_{ntot}')
        file_name = replace_periods(file_name) + '.png'
        f.savefig(figures_directory + file_name)

In [ ]:
def plot_snmd(
        nucleon, nucleus_name, kvnn, channels=('1S0','3S1'), kmax=15.0,
        kmid=3.0, ntot=120, generator='Wegner', lamb=1.35, density='SLy4',
        x_limits=(0,6), y_limits=(4e-4,3e2), xlabel_size=16, ylabel_size=16,
        show_low_q=False, save=False):
    """
    """
    
    # --- Figure set-up --- #
    plt.close('all')
    f, ax = plt.subplots(figsize=(4,4))
    
    # --- Plot final features --- #
    
    # --- Legend --- #
    ax.legend(loc='upper right', fontsize=16, frameon=False)
    
    # --- Save file --- #
    if save:
        file_name = (f'{nucleon}_momentum_distribution_{nucleus_name}'
                     f'_kvnn_{kvnn}')
        for ichannel in channels:
            file_name += f'_{ichannel}'
        file_name += (f'_kmax_{kmax}_kmid_{kmid}_ntot_{ntot}_{generator}_lamb'
                      f'_{lamb}_{density}')
        file_name = replace_periods(file_name) + '.png'
        f.savefig(figures_directory + file_name)

In [ ]:
def plot_multiple_snmd(
        nucleon, nuclei, kvnn, channels=('1S0','3S1'), kmax=15.0, kmid=3.0,
        ntot=120, generator='Wegner', lamb=1.35, density='SLy4',
        x_limits=(2,6), y_limits=(4e-4,3e2), xlabel_size=16, ylabel_size=16,
        show_low_q=False, save=False):
    """
    """
    
    # --- Figure set-up --- #
    plt.close('all')
    col_number = len(nuclei)
    f, axs = plt.subplots(1, col_number, sharex=True, sharey=True,
                          figsize=(4*col_number,4))
    
    # --- Plot final features --- #
    
    # --- Adjust axes --- #
    # Amount of white space in-between sub-plots
    f.subplots_adjust(wspace=0.05)
    
    # --- Legend --- #
    ax.legend(loc='upper right', fontsize=14, frameon=False)
    
    # --- Save file --- #
    if save:
        file_name = f'{nucleon}_momentum_distributions'
        for inucleus in nuclei:
            file_name += f'_{inucleus[0]}'
        file_name += f'_kvnn_{kvnn}'
        for ichannel in channels:
            file_name += f'_{ichannel}'
        file_name += (f'_kmax_{kmax}_kmid_{kmid}_ntot_{ntot}_{generator}_lamb'
                      f'_{lamb}_{density}')
        file_name = replace_periods(file_name) + '.png'
        f.savefig(figures_directory + file_name)

In [ ]:
def plot_snmd_tails(
        nucleon, nuclei, kvnn, channels=('1S0','3S1'), kmax=15.0, kmid=3.0,
        ntot=120, generator='Wegner', lamb=1.35, density='SLy4',
        x_limits=(1.5,6.0), y_limits=(1e-3,1e0), xlabel_size=16,
        ylabel_size=16, save=False):
    """
    """
    
    # --- Figure set-up --- #
    plt.close('all')
    f, ax = plt.subplots(figsize=(4,4))
    
    # --- Legend --- #
    ax.legend(loc='upper right', fontsize=13, frameon=False)
    
    # --- Save file --- #
    if save:
        file_name = f'{nucleon}_momentum_distribution_tails'
        for inucleus in nuclei:
            file_name += f'_{inucleus[0]}'
        file_name += f'_kvnn_{kvnn}'
        for ichannel in channels:
            file_name += f'_{ichannel}'
        file_name += (f'_kmax_{kmax}_kmid_{kmid}_ntot_{ntot}_{generator}_lamb'
                      f'_{lamb}_{density}')
        file_name = replace_periods(file_name) + '.png'
        f.savefig(figures_directory + file_name)

In [ ]:
def snmd_partial_wave_contributions(
        nucleon, nuclei, kvnn, channels=('1S0','3S1'), kmax=15.0, kmid=3.0,
        ntot=120, generator='Wegner', lamb=1.35, density='SLy4'):
    """
    """
    return None

In [ ]:
def proton_over_neutron_pmd_ratios(
        nuclei, kvnn, channels=('1S0','3S1'), kmax=15.0, kmid=3.0,
        ntot=120, generator='Wegner', lamb=1.35, density='SLy4',
        x_limits=(1.5,4), y_limits=(0.82,1.15), xlabel_size=16,
        ylabel_size=16, ratio_label=True, save=False):
    """
    """
    
    # --- Figure set-up --- #
    plt.close('all')
    f, ax = plt.subplots(figsize=(4,4))
    
    # --- Legend --- #
    ax.legend(loc='upper right', fontsize=12, frameon=False, ncol=2)
    
    # --- Additional labels --- #
    if ratio_label:  # Label (pp+pn)/(nn+np)
        ax.add_artist(
            AnchoredText(r'$\frac{pp+pn}{nn+np}$', loc='lower left',
                         prop=dict(size=24), frameon=False)
        )
    
    # --- Save file --- #
    if save:
        file_name = 'proton_over_neutron_pmd_ratios'
        for inucleus in nuclei:
            file_name += f'_{inucleus[0]}'
        file_name += f'_kvnn_{kvnn}'
        for ichannel in channels:
            file_name += f'_{ichannel}'
        file_name += (f'_kmax_{kmax}_kmid_{kmid}_ntot_{ntot}_{generator}_lamb'
                      f'_{lamb}_{density}')
        file_name = replace_periods(file_name) + '.png'
        f.savefig(figures_directory + file_name)

In [ ]:
def pp_over_pn_pmd_ratios(
        nuclei, kvnn, channels=('1S0','3S1'), kmax=15.0, kmid=3.0,
        ntot=120, generator='Wegner', lamb=1.35, density='SLy4',
        x_limits=(1.5,4), y_limits=(-0.01,0.25), xlabel_size=16,
        ylabel_size=16, ratio_label=True, save=False):
    """
    """
    
    # --- Figure set-up --- #
    plt.close('all')
    f, ax = plt.subplots(figsize=(4,4))
    
    # --- Legend --- #
    ax.legend(loc='upper left', fontsize=14, frameon=False)
    
    # --- Additional labels --- #
    if ratio_label:  # Label pp/(pn+np)
        ax.add_artist(
            AnchoredText(r'$\frac{pp}{pn+np}$', loc='upper right',
                         prop=dict(size=25), frameon=False)
        )
    
    # --- Save file --- #
    if save:
        file_name = 'pp_over_pn_pmd_ratios'
        for inucleus in nuclei:
            file_name += f'_{inucleus[0]}'
        file_name += f'_kvnn_{kvnn}'
        for ichannel in channels:
            file_name += f'_{ichannel}'
        file_name += (f'_kmax_{kmax}_kmid_{kmid}_ntot_{ntot}_{generator}_lamb'
                      f'_{lamb}_{density}')
        file_name = replace_periods(file_name) + '.png'
        f.savefig(figures_directory + file_name)

In [ ]:
def plot_src_scaling_factors(
        kvnn, lamb, channels=('1S0','3S1'), kmax=15.0, kmid=3.0, ntot=120,
        generator='Wegner', x_limits=(3e0,3e2),
        y_limits=(0.0, 6.5), xlabel_size=16, ylabel_size=16,
        lambda_label=True, save=False):
    """
    Note, this function automatically uses SLy4 and Gogny densities.
    """
    
    # --- Figure set-up --- #
    plt.close('all')
    f, ax = plt.subplots(figsize=(4,4))
    
    # --- Legend --- #
    ax.legend(loc='lower right', fontsize=16, frameon=False)
    
    # --- Additional labels --- #
    if lambda_label:  # Label \lambda
        ax.add_artist(
            AnchoredText(label_lambda(lamb), loc='upper left',
                         prop=dict(size=18), frameon=False)
        )
    
    # --- Save file --- #
    if save:
        file_name = f'src_scaling_factors_kvnn_{kvnn}_lamb_{lamb}'
        for ichannel in channels:
            file_name += f'_{ichannel}'
        file_name += (f'_kmax_{kmax}_kmid_{kmid}_ntot_{ntot}_{generator}'
                      '_densities_SLy4_Gogny')
        file_name = replace_periods(file_name) + '.png'
        f.savefig(figures_directory + file_name)

## $^3\rm{S}_1$ to $^1\rm{S}_0$ ratio of SRG transformations

## Deuteron bar chart

## Deuteron momentum distribution

## Proton momentum distributions

## Pair momentum distribution ratios

## SRC scaling factors $a_2$